In [0]:
from google.colab import files
uploaded = files.upload()
print("OK")

Saving winequality-data.csv to winequality-data (1).csv
OK


In [0]:
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import SGDClassifier

  
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.model_selection
import sklearn.preprocessing
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_validate

In [0]:
df = pd.read_csv('winequality-data.csv')
print(df.shape)

In [0]:
# df = df.groupby('quality').head(1200)

fig = plt.figure(figsize=(8,6))
df.groupby('quality').id.count().plot.bar(ylim=0)
plt.show()



In [0]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(df.drop(['id','quality'],axis = 1),
                                                                            df['quality'],
                                                                            test_size = 0.3,
                                                                            random_state = 42)


In [0]:
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)

In [0]:
model = keras.Sequential([
    keras.layers.Dense(12, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [0]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
model.fit(X_train_scaled, y_train.values, epochs=100)

In [0]:
X_test_scaled = min_max_scaler.fit_transform(X_test)
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(test_loss, test_acc)

print('Test accuracy:', test_acc, 'Test loss:', test_loss)

# x1 = X_test_scaled[445]
# y1 = y_test.values[445]
# x1 = (np.expand_dims(x1,0))
# print('Expected:', y1, '; Got:', np.argmax(prediction))

y_pred = model.predict(X_test_scaled)
labels = []

for x in y_pred:
  labels.append(np.argmax(x))
  
print(type(y_test.values))
confusion_matrix(y_test.values, np.asarray(labels))
y_test.name = 'Actually'

pd.crosstab(y_test, pd.Series(np.asarray(labels),name='Predicted'), margins = 'True')

In [0]:
X = df.drop(['id','quality'],axis=1).values
Y = df['quality'].values     
X_new = SelectKBest(chi2, k=9).fit_transform(X, Y)

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_new, Y, test_size = 0.3, random_state = 42)
X_train_scaled = min_max_scaler.fit_transform(X_train)

model = keras.Sequential([
    keras.layers.Dense(12, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs=50)


In [0]:
X_test_scaled = min_max_scaler.fit_transform(X_test)
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(test_loss, test_acc)

In [0]:
# fixed acidity isn't varies too little in correspondance with wine quality
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'fixed.acidity', data = df)


In [0]:
# volatile acidity goes up when the quality is better
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'volatile.acidity', data = df)


In [0]:
# citric acidity doesn't vary too much
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'citric.acid', data = df)


In [0]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'residual.sugar', data = df)


In [0]:
# chlorides go down when the quality gets better
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'chlorides', data = df)


In [0]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'free.sulfur.dioxide', data = df)


In [0]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'total.sulfur.dioxide', data = df)


In [0]:
# sulphates value doesn't vary too much
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'sulphates', data = df)


In [0]:
# good wines have more alcohol
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'alcohol', data = df)


In [0]:
#X = df.drop(['id','quality','fixed.acidity','citric.acid','sulphates'],axis=1).values
Y = df['quality'].values
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_new, Y, test_size = 0.7, random_state = 42)
X_train_scaled = min_max_scaler.fit_transform(X_train)

model = keras.Sequential([
    keras.layers.Dense(12, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs=50)


In [0]:
X_test_scaled = min_max_scaler.fit_transform(X_test)
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(test_loss, test_acc)
y_pred = model.predict(X_test_scaled)
labels = []

for x in y_pred:
  labels.append(np.argmax(x))
  
confusion_matrix(y_test, np.asarray(labels))

pd.crosstab(pd.Series(y_test,name='Actually'), pd.Series(np.asarray(labels),name='Predicted'), margins = 'True')

In [0]:
bins = (1, 6, 8, 10)
group_names = ['poor','medium','high']
df['quality'] = pd.cut(df['quality'], bins = bins, labels = group_names)

label_quality = LabelEncoder()

df['quality'] = label_quality.fit_transform(df['quality'])

df['quality'].value_counts()

2    3066
1     847
0       5
Name: quality, dtype: int64

In [0]:
sns.countplot(df['quality'])

X_train, X_test, y_train, y_test =  sklearn.model_selection.train_test_split(df.drop('quality', axis = 1), df['quality'], test_size = 0.3, random_state = 42)

X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)

In [0]:
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)

In [0]:
print(classification_report(y_test, pred_rfc))
print(confusion_matrix(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.62      0.16      0.25       260
           2       0.80      0.97      0.88       914

   micro avg       0.79      0.79      0.79      1176
   macro avg       0.47      0.38      0.38      1176
weighted avg       0.76      0.79      0.74      1176

[[  0   2   0]
 [  0  41 219]
 [  0  23 891]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
sgd = SGDClassifier(penalty=None)
sgd.fit(X_train, y_train)
pred_sgd = sgd.predict(X_test)

In [0]:
print(classification_report(y_test, pred_sgd))
print(confusion_matrix(y_test, pred_sgd))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.53      0.42      0.47       260
           2       0.85      0.90      0.87       914

   micro avg       0.79      0.79      0.79      1176
   macro avg       0.46      0.44      0.45      1176
weighted avg       0.77      0.79      0.78      1176

[[  0   2   0]
 [  0 110 150]
 [  0  95 819]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
svc = SVC()
svc.fit(X_train, y_train)
pred_svc = svc.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [0]:
print(classification_report(y_test, pred_svc))
print(confusion_matrix(y_test,pred_svc))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00       260
           2       0.78      1.00      0.87       914

   micro avg       0.78      0.78      0.78      1176
   macro avg       0.26      0.33      0.29      1176
weighted avg       0.60      0.78      0.68      1176

[[  0   0   2]
 [  0   0 260]
 [  0   0 914]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
param = {
    'C': [0.1,0.8,0.9,1,1.1,1.2,1.3,1.4],
    'kernel':['linear', 'rbf'],
    'gamma' :[0.1,0.8,0.9,1,1.1,1.2,1.3,1.4]
}
grid_svc = GridSearchCV(svc, param_grid=param, scoring='accuracy', cv=10)

In [0]:
grid_svc.fit(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4], 'kernel': ['linear', 'rbf'], 'gamma': [0.1, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [0]:
grid_svc.best_params_

{'C': 1.4, 'gamma': 1.4, 'kernel': 'rbf'}

In [0]:
svc = SVC(C = 1.4, gamma = 1.4, kernel = 'rbf')
svc.fit(X_train, y_train)
pred_svc = svc.predict(X_test)

pred_svc = svc.predict(X_test)
print(classification_report(y_test, pred_svc))
print(confusion_matrix(y_test,pred_svc))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.64      0.22      0.33       260
           2       0.81      0.97      0.88       914

   micro avg       0.80      0.80      0.80      1176
   macro avg       0.48      0.40      0.40      1176
weighted avg       0.77      0.80      0.76      1176

[[  0   2   0]
 [  0  57 203]
 [  0  30 884]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
rfc_eval = cross_val_score(estimator = rfc, X = X_train, y = y_train, cv = 10)
rfc_eval.mean()


In [0]:
sgd_eval = cross_val_score(estimator = sgd, X = X_train, y = y_train, cv = 10)
sgd_eval.mean()

In [0]:
X_train, X_test, y_train, y_test =  sklearn.model_selection.train_test_split(df.drop('quality', axis = 1), df['quality'], test_size = 0.3, random_state = 7)

X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)

rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)

print(classification_report(y_test, pred_rfc))
print(confusion_matrix(y_test, pred_rfc))

In [0]:
rfc_eval = cross_validate(estimator = sgd, X = X_train, y = y_train, cv = 100)

In [0]:
rfc_eval['test_score'].mean()

0.7878857030581169